In [ ]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

ROM_SIZE = 0x2000 #8K

SOC_UP = 0x0000;
SOC_LA = 0x1000;
PL_AA_MB = 0x2000;
PL_AA = 0x2100;
SOC_IS = 0x3000;
SOC_AS = 0x4000;
SOC_CC = 0x5000;
PL_AS = 0x6000;
PL_IS = 0x7000;
PL_DMA = 0x8000;
# add userdma base address
PL_UPDMA = 0x9000;

In [ ]:
###ol = Overlay("/home/xilinx/jupyter_notebooks/PS/caravel_fpga.bit")
ol = Overlay("caravel_fpga.bit") # modified
#ol.ip_dict

In [ ]:
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
# ipReadROMCODE = ol.read_romcode_0

#Add for SPI
ip_QSPI = ol.axi_quad_spi_0

In [ ]:
# ============================================
# AXI QuadSPI Control
# ============================================
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04
XSP_CR_CLK_POLARITY_MASK = 0x08
XSP_CR_CLK_PHASE_MASK = 0x10
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def write_tx_fifo(AxiQspi):
    #print("TransferData")
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    StatusReg = AxiQspi.read(XSP_SR_OFFSET)
    while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
        StatusReg = AxiQspi.read(XSP_SR_OFFSET)

    #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)


def read_rx_fifo(bypass_length, AxiQspi):
    #print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    
    # By pass the FIFO data during master issue command and address to slave device
    command_addr_length = bypass_length
    counter = 0    
    
    while RxFifoStatus == 0:
        #temp = AxiQspi.read(XSP_RFO_OFFSET)
        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        #print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    

        counter = counter + 1
        if(counter > command_addr_length):
            resp.append(temp)        
        
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp

In [ ]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))


In [ ]:
# ============================================
# Release Reset First before passthrough mode
# ============================================
# Release Caravel reset
# 0x10 : Data signal of outpin_ctrl
#        bit 0  - outpin_ctrl[0] (Read/Write)
#        others - reserved
print (ipOUTPIN.read(0x10))
ipOUTPIN.write(0x10, 1)
print (ipOUTPIN.read(0x10))

In [ ]:
# ============================================
# Load firmware (fsic.hex) to memory npROM
# ============================================

# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)

npROM_index = 0
npROM_offset = 0
###fiROM = open("/home/xilinx/jupyter_notebooks/PS/fsic.hex", "r+")
fiROM = open("fsic.hex", "r+") # modified

for line in fiROM:
    # offset header
    if line.startswith('@'):
        # Ignore first char @
        npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
        npROM_offset = npROM_offset >> 2 # 4byte per offset
        #print (npROM_offset)
        npROM_index = 0
        continue
    #print (line)

    # We suppose the data must be 32bit alignment
    buffer = 0
    bytecount = 0
    for line_byte in line.strip(b'\x00'.decode()).split():
        buffer += int(line_byte, base = 16) << (8 * bytecount)
        bytecount += 1
        # Collect 4 bytes, write to npROM
        if(bytecount == 4):
            npROM[npROM_offset + npROM_index] = buffer
            # Clear buffer and bytecount
            buffer = 0
            bytecount = 0
            npROM_index += 1
            #print (npROM_index)
            continue
    # Fill rest data if not alignment 4 bytes
    if (bytecount != 0):
        npROM[npROM_offset + npROM_index] = buffer
        npROM_index += 1
    
fiROM.close()

In [ ]:
# ============================================
# Enabling passthrou mode
# ============================================
cnfg(ip_QSPI)
# Passthrou mode - Write command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x02) # Command: Write data to memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
write_tx_fifo(ip_QSPI)

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

In [ ]:
# ============================================
# Writing FW into SPIROM
# ============================================
# Fill up Tx_FIFO (16) for each write_tx_fifo
for index in range (ROM_SIZE >> 2):
     # 4 bytes alignment in npROM
    for byte_shift in range(4):
        tmp = int((npROM[index] >> (byte_shift * 8)) & 0xFF)
        ip_QSPI.write(XSP_DTR_OFFSET, tmp) # Write_data
    # TX_FIFO = 16, 4 * 4 = 16
    if((index % 3) == 3):
        write_tx_fifo(ip_QSPI)
        
# If rest data is not enough 16 bytes. Tx_FIFO is not empty
    StatusReg = ip_QSPI.read(XSP_SR_OFFSET)
    if ((StatusReg & XSP_SR_TX_EMPTY_MASK) == 0):
         write_tx_fifo(ip_QSPI)

In [ ]:
# ============================================
# Read SPIROM for testing
# ============================================
cnfg(ip_QSPI)

In [ ]:
# Test Passthrou mode - Read command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x03) # Command: Read data from memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)

In [ ]:
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(5, ip_QSPI)
for data in rx_final:
    print (hex(data))

In [ ]:
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

In [ ]:
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(0, ip_QSPI)
for data in rx_final:
    print (hex(data))

In [ ]:
# ============================================
# Exit passthrou mode, FW will be fetched
# ============================================
ip_QSPI.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

In [ ]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))

In [ ]:
IP_BASE_ADDRESS = 0x60000000
###ADDRESS_RANGE = 0x9000
ADDRESS_RANGE = 0xa000 # modified
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# PL_FSIC Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# PL_IS Config
ADDRESS_OFFSET = PL_IS #0x7000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x12345671)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x12345673)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# PL_DMA Config
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
# PL_AS Config
ADDRESS_OFFSET = PL_AS # 0x6000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345676)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# PL_AA Config
ADDRESS_OFFSET = PL_AA # 0x2100
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04))) 

In [ ]:
mmio.write(ADDRESS_OFFSET+0x00, 0x11111111)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222222)

In [ ]:
# PL_AA_MB Mailbox
ADDRESS_OFFSET = PL_AA_MB # 0x2000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))
print("mmio.read(ADDRESS_OFFSET+0x08): ", hex(mmio.read(ADDRESS_OFFSET+0x08)))
print("mmio.read(ADDRESS_OFFSET+0x0C): ", hex(mmio.read(ADDRESS_OFFSET+0x0C)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x14): ", hex(mmio.read(ADDRESS_OFFSET+0x14)))
print("mmio.read(ADDRESS_OFFSET+0x18): ", hex(mmio.read(ADDRESS_OFFSET+0x18)))
print("mmio.read(ADDRESS_OFFSET+0x1C): ", hex(mmio.read(ADDRESS_OFFSET+0x1C)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x11111112)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222223)
mmio.write(ADDRESS_OFFSET+0x08, 0x33333334)
mmio.write(ADDRESS_OFFSET+0x0C, 0x44444445)
mmio.write(ADDRESS_OFFSET+0x10, 0x55555556)
mmio.write(ADDRESS_OFFSET+0x14, 0x66666667)
mmio.write(ADDRESS_OFFSET+0x18, 0x77777778)
mmio.write(ADDRESS_OFFSET+0x1C, 0x88888889)

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# PL_Caravel Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# Caravel-IS Config
ADDRESS_OFFSET = SOC_IS # 0x3000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-IS Config
ADDRESS_OFFSET = 0x3000
mmio.write(ADDRESS_OFFSET, 0x00000003)

In [ ]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
mmio.write(ADDRESS_OFFSET, 0x00000006)

In [ ]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000)

In [ ]:
# Caravel-LA Config
ADDRESS_OFFSET = SOC_LA # 0x1000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-UP Config
ADDRESS_OFFSET = SOC_UP # 0x0000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# LADMA Verification
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
### # Allocation memory
### ladma_buf = allocate(shape=(1024,), dtype=np.uint32)
### print("ladma_buf.device_address: ", hex(ladma_buf.device_address))
### 
### IP_BASE_ADDRESS = ladma_buf.device_address
### ADDRESS_RANGE = 0x1000
### buf_mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
### # 0x00 : Control signals
### #        bit 0 - ap_start (R/W/COH)
### #        bit 1 - ap_done (R/COR)
### #        bit 2 - ap_idle (R)
### #        bit 3 - ap_ready (R/COR)
### # 0x10 : Buffer transfer done status register
### #        bit 0 - buffer transfer done status (R)
### # 0x20 : Buffer transfer done status clear register
### #        bit 0 - clear buffer transfer done status (R/W)
### # 0x28 : Buffer Length
### #        bit 31~0 - set buffer length (must 1024)
### # 0x30 : Triggered condition 
### #        bit 23~0 - set triggered condidtion (R/W)
### #        others  - reserved
### # 0x34 : Buffer Lower base address 
### #        bit 31~0 - (R/W)
### # 0x38 : Buffer High base address
### #        bit 31~0 - (R/W)
### # ladma Configuration
### ADDRESS_OFFSET = PL_DMA # 0x8000
### # exit clear operation
### mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
### # set buffer length
### mmio.write(ADDRESS_OFFSET + 0x28, 0x00000400)
### # set trigger condition
### mmio.write(ADDRESS_OFFSET + 0x30, 0x00000000)
### # set buffer low
### mmio.write(ADDRESS_OFFSET + 0x38, ladma_buf.device_address)
### # set buffer high
### mmio.write(ADDRESS_OFFSET + 0x3C, 0x00000000)

In [ ]:
### ADDRESS_OFFSET = PL_DMA # 0x8000
### print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
### print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
### print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
### print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
### print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
### print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
### print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
### # ladma Configuration
### ADDRESS_OFFSET = PL_DMA # 0x8000
### # set ap_start
### mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)

In [ ]:
### # enable la 0xFFFFFF
### ADDRESS_OFFSET = SOC_LA # 0x1000
### mmio.write(ADDRESS_OFFSET, 0x00FFFFFF)
### #print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
### 
### # select target UP
### ADDRESS_OFFSET = SOC_CC # 0x5000
### mmio.write(ADDRESS_OFFSET, 0x00000003)
### #print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
### # ladma Configuration
### ADDRESS_OFFSET = PL_DMA # 0x8000
### while True:
###     if mmio.read(ADDRESS_OFFSET+0x10) == 0x01:
###         break
### print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
### 
### # disable la 0x000000
### ADDRESS_OFFSET = SOC_LA # 0x1000
### mmio.write(ADDRESS_OFFSET, 0x00000000)
### # select fake UP
### ADDRESS_OFFSET = SOC_CC # 0x5000
### mmio.write(ADDRESS_OFFSET, 0x00000000) 
### 
### ADDRESS_OFFSET = PL_DMA # 0x8000
### # clear buffer transfer done operation
### mmio.write(ADDRESS_OFFSET + 0x20, 0x00000001)
### # set ap_start
### mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)
### while True:
###     if mmio.read(ADDRESS_OFFSET+0x10) != 0x01:
###         break
### print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))      

In [ ]:
### ADDRESS_OFFSET = PL_DMA # 0x8000
### print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
### print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
### print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
### print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
### print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
### print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
### print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
### #dump la log to file
### file = open("simulate.log", "w")
### for i in range(0,0xFFF,4):
###     file.write('{:08x}'.format(buf_mmio.read(i))+"\n")
### file.close()

In [ ]:
### # ladma Configuration
### ADDRESS_OFFSET = PL_DMA # 0x8000
### # exit clear operation
### mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
### # set ap_start again
### mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)
### 
### # Confirming SOC_CC & SOC_LA Reading is still work
### ADDRESS_OFFSET = SOC_CC # 0x5000
### print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
### print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
### # Translate simulate.log to output_new.vcd, which can be open by gtakwave
### %run read_la_data.py

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# OpticalFlow IP + userDMA Verification (Added by us)
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# Check Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
# select user project 2
mmio.write(ADDRESS_OFFSET, 0x00000002)

In [ ]:
# Check new Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

# The result should be "2" !!!

In [ ]:
# Caravel-UP Config (configure OpticalFlow IP)
ADDRESS_OFFSET = SOC_UP # 0x0000

In [ ]:
# Set rst = 1 (address=0x00)
offset = 0x0
write_data = 0x1
mmio.write(ADDRESS_OFFSET + offset, write_data)

In [ ]:
# Read back & check
read_value = mmio.read(ADDRESS_OFFSET + offset)
if read_value == write_data:
    print("[PASS] Read OpticalFlow address ", hex(ADDRESS_OFFSET + offset), ": read out data = ", hex(read_value), "; golden value = ", hex(write_data))
else:
    print("[ERROR] Read OpticalFlow address ", hex(ADDRESS_OFFSET + offset), ": read out data = ", hex(read_value), "; golden value = ", hex(write_data))

In [ ]:
# Set rst = 0 (address=0x00)
offset = 0x0
write_data = 0x0
mmio.write(ADDRESS_OFFSET + offset, write_data)

In [ ]:
# Read back & check
read_value = mmio.read(ADDRESS_OFFSET + offset)
if read_value == write_data:
    print("[PASS] Read OpticalFlow address ", hex(ADDRESS_OFFSET + offset), ": read out data = ", hex(read_value), "; golden value = ", hex(write_data))
else:
    print("[ERROR] Read OpticalFlow address ", hex(ADDRESS_OFFSET + offset), ": read out data = ", hex(read_value), "; golden value = ", hex(write_data))

In [ ]:
TST_FRAME_WIDTH = 1024
TST_FRAME_HEIGHT = 436
shift_threshold = 105

# Program widthIn, heightIn, and shift_threshold
mmio.write(ADDRESS_OFFSET + 0x04, TST_FRAME_WIDTH)
mmio.write(ADDRESS_OFFSET + 0x08, TST_FRAME_HEIGHT)
mmio.write(ADDRESS_OFFSET + 0x0C, shift_threshold)

In [ ]:
# Read back widthIn, heightIn, shift_threshold & check
print("Read OpticalFlow address ", hex(ADDRESS_OFFSET + 0x04), ": read out data = ", hex(mmio.read(ADDRESS_OFFSET + 0x04)))
print("Read OpticalFlow address ", hex(ADDRESS_OFFSET + 0x08), ": read out data = ", hex(mmio.read(ADDRESS_OFFSET + 0x08)))
print("Read OpticalFlow address ", hex(ADDRESS_OFFSET + 0x0C), ": read out data = ", hex(mmio.read(ADDRESS_OFFSET + 0x0C)))

In [ ]:

# Program ap_start = 1
###mmio.write(ADDRESS_OFFSET, 0x00000001)

In [ ]:
test_length = TST_FRAME_WIDTH*TST_FRAME_HEIGHT

In [ ]:
# Allocation userDMA s2m memory
updma_buf_s2m = allocate(shape=(TST_FRAME_WIDTH*TST_FRAME_HEIGHT*3,), dtype=np.uint32)
print("updma_buf_s2m.device_address: ", hex(updma_buf_s2m.device_address))

s2m_BASE_ADDRESS = updma_buf_s2m.device_address
s2m_ADDRESS_RANGE = 0x1000
s2m_buf_mmio = MMIO(s2m_BASE_ADDRESS, s2m_ADDRESS_RANGE)

# userDMA Configuration
ADDRESS_OFFSET = PL_UPDMA # 0x9000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
# disable to clear
mmio.write(ADDRESS_OFFSET + 0x30, 0x00000000)
# set buffer length
mmio.write(ADDRESS_OFFSET + 0x28, test_length*3)
# set buffer low
mmio.write(ADDRESS_OFFSET + 0x38, updma_buf_s2m.device_address)
# set buffer high
mmio.write(ADDRESS_OFFSET + 0x3C, 0x00000000)

In [ ]:
ADDRESS_OFFSET = PL_UPDMA # 0x9000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
# Allocation userDMA m2s memory
updma_buf_m2s = allocate(shape=(TST_FRAME_WIDTH*TST_FRAME_HEIGHT,), dtype=np.uint32)
print("updma_buf_m2s.device_address: ", hex(updma_buf_m2s.device_address))

# set input data
frame1_pointer = open("frame1.hex", "r")
frame2_pointer = open("frame2.hex", "r")
frame3_pointer = open("frame3.hex", "r")
frame4_pointer = open("frame4.hex", "r")
for i in range(TST_FRAME_WIDTH*TST_FRAME_HEIGHT):
    frame1_value = int(frame1_pointer.readline().strip('\n'),16)
    frame2_value = int(frame2_pointer.readline().strip('\n'),16)
    frame3_value = int(frame3_pointer.readline().strip('\n'),16)
    frame4_value = int(frame4_pointer.readline().strip('\n'),16)
    updma_buf_m2s[i] = (frame4_value<<24) + (frame3_value<<16) + (frame2_value<<8) + frame1_value
    # print(hex((frame4_value<<24) + (frame3_value<<16) + (frame2_value<<8) + frame1_value))

frame1_pointer.close()
frame2_pointer.close()
frame3_pointer.close()
frame4_pointer.close()

m2s_BASE_ADDRESS = updma_buf_m2s.device_address
m2s_ADDRESS_RANGE = 0x1000
m2s_buf_mmio = MMIO(m2s_BASE_ADDRESS, m2s_ADDRESS_RANGE)

# userDMA Configuration
ADDRESS_OFFSET = PL_UPDMA # 0x9000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x78, 0x00000000)
# disable to clear
mmio.write(ADDRESS_OFFSET + 0x88, 0x00000000)
# set buffer length
mmio.write(ADDRESS_OFFSET + 0x80, TST_FRAME_WIDTH*TST_FRAME_HEIGHT)
# set buffer low
mmio.write(ADDRESS_OFFSET + 0x5C, updma_buf_m2s.device_address)
# set buffer high
mmio.write(ADDRESS_OFFSET + 0x60, 0x00000000)

In [ ]:
ADDRESS_OFFSET = PL_UPDMA # 0x9000
print("mmio.read(ADDRESS_OFFSET+0x78): ", hex(mmio.read(ADDRESS_OFFSET+0x78)))
print("mmio.read(ADDRESS_OFFSET+0x88): ", hex(mmio.read(ADDRESS_OFFSET+0x88)))
print("mmio.read(ADDRESS_OFFSET+0x80): ", hex(mmio.read(ADDRESS_OFFSET+0x80)))
print("mmio.read(ADDRESS_OFFSET+0x5C): ", hex(mmio.read(ADDRESS_OFFSET+0x5C)))
print("mmio.read(ADDRESS_OFFSET+0x60): ", hex(mmio.read(ADDRESS_OFFSET+0x60)))

In [ ]:
# PL_UPDMA, set ap_start
mmio.write(PL_UPDMA, 0x00000001)

In [ ]:
# Check userDMA Done
keepChk = True
while keepChk:
    if mmio.read(PL_UPDMA + 0x10) == 0x00000001:
        keepChk = False
print("[Congradulations] Read userDMA buffer transfer done, address ",hex(PL_UPDMA + 0x10), ": read out data = 1")

In [ ]:
ADDRESS_OFFSET = PL_UPDMA # 0x9000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
# print out updma_buf_s2m
for i in range(test_length*3):
    print("updma_buf_s2m[", i, "] = ", hex(updma_buf_s2m[i]))